In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [20]:
test_dataset

In [3]:
class Branched_CNN(nn.Module):

    def __init__(self, in_channels):
        super(Branched_CNN, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3, branch_pool]
        return torch.cat(outputs, 1)

In [4]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.network1 = Branched_CNN(in_channels=10)
        self.network2 = Branched_CNN(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.network1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.network2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [5]:
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [10]:
model.parameters

<bound method Module.parameters of Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(88, 20, kernel_size=(5, 5), stride=(1, 1))
  (network1): Branched_CNN(
    (branch1x1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3dbl_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch3x3dbl_2): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch3x3dbl_3): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch_pool): Conv2d(10, 24, kernel_size=(1, 1), stride=(1, 1))
  )
  (network2): Branched_CNN(
    (branch1x1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3dbl_1)

In [11]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [12]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:
for epoch in range(1, 3):
    train(epoch)
    test()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305039
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.305316
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.291271
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.301392
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.293334
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.304334
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.303862
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.302210
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.281712
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.303314
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.289787
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.263515
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.299057
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.291286
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.285590
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.283137
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.274502
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.250583
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.234917
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.250358
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.201009
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.034796
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.162235
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.188865
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.030589
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.143383
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.138169
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.053236
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.123104
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.066866
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.119778
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.106021
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.144926
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.116009
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.054208
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.095624
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.199365
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.147326
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.084900
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.089233
